In [18]:
import pandas as pd
import numpy as np
pd.__version__

'2.2.3'

In [19]:
!pip install pyarrow


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [20]:
df = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet')

In [21]:
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00


In [22]:
import sklearn
sklearn.__version__
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression


### Homework begins here

In [23]:
# 1. Calculating the number of columns
n_cols = len(df.columns)
print(f"Number of columns in the dataset is: {n_cols}")

Number of columns in the dataset is: 19


In [24]:
# 2. Now let's compute the duration variable. It should contain the duration of a ride in minutes. 
# What's the standard deviation of the trips duration in January?

df['duration'] = (df.tpep_dropoff_datetime - df.tpep_pickup_datetime).dt.total_seconds()/60
std_duration = df.duration.std()
# Converting to minutes
print(f"Standard deviation of the trips duration in January is: {std_duration}")

Standard deviation of the trips duration in January is: 42.59435124195458


In [25]:
# 3. What fraction of the records left after you dropped the outliers?
new_df = df[(df.duration >= 1) & (df.duration<= 60)]
fraction_left = (len(new_df)/len(df))*100
print(f"Fraction of the data left after filtering is: {fraction_left}")

Fraction of the data left after filtering is: 98.1220282212598


In [26]:
new_df["PULocationID"] = new_df["PULocationID"].astype(str)
new_df["DOLocationID"] = new_df["DOLocationID"].astype(str)

features = new_df[['PULocationID', 'DOLocationID']].to_dict(orient='records')
dv = DictVectorizer()
X = dv.fit_transform(features)


/tmp/ipykernel_771/2968361064.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df["PULocationID"] = new_df["PULocationID"].astype(str)
/tmp/ipykernel_771/2968361064.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df["DOLocationID"] = new_df["DOLocationID"].astype(str)


In [27]:
print(f"Dimensionality of X: {X.shape[1]}")

Dimensionality of X: 515


In [28]:
linreg = LinearRegression()
y = new_df.duration
model = linreg.fit(X,y)

y_pred = model.predict(X)

from sklearn.metrics import root_mean_squared_error
rmse = root_mean_squared_error(y, y_pred)
print(f"RMSE: {rmse:.3f}")

RMSE: 7.649


In [29]:
val_df = pd.read_parquet("https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet")
val_df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
0,1,2023-02-01 00:32:53,2023-02-01 00:34:34,2.0,0.30,1.0,N,142,163,2,4.4,3.50,0.5,0.0,0.0,1.0,9.40,2.5,0.00
1,2,2023-02-01 00:35:16,2023-02-01 00:35:30,1.0,0.00,1.0,N,71,71,4,-3.0,-1.00,-0.5,0.0,0.0,-1.0,-5.50,0.0,0.00
2,2,2023-02-01 00:35:16,2023-02-01 00:35:30,1.0,0.00,1.0,N,71,71,4,3.0,1.00,0.5,0.0,0.0,1.0,5.50,0.0,0.00
3,1,2023-02-01 00:29:33,2023-02-01 01:01:38,0.0,18.80,1.0,N,132,26,1,70.9,2.25,0.5,0.0,0.0,1.0,74.65,0.0,1.25
4,2,2023-02-01 00:12:28,2023-02-01 00:25:46,1.0,3.22,1.0,N,161,145,1,17.0,1.00,0.5,3.3,0.0,1.0,25.30,2.5,0.00


In [30]:
val_df['duration'] = (val_df.tpep_dropoff_datetime - val_df.tpep_pickup_datetime).dt.total_seconds() / 60

val_df["PULocationID"] = val_df["PULocationID"].astype(str)
val_df["DOLocationID"] = val_df["DOLocationID"].astype(str)

val_df = val_df[(val_df['duration'] >= 1) & (val_df['duration'] <= 60)]

features = val_df[['PULocationID', 'DOLocationID']].to_dict(orient='records')

X_new = dv.transform(features)  # DO NOT fit again!

y_new = val_df['duration']
y_pred2 = model.predict(X_new)


In [31]:
y_pred2 = model.predict(X_new)

In [32]:
rmse = root_mean_squared_error(y_new, y_pred2)
print(f"The RMSE is: {rmse}")

The RMSE is: 7.811818743246608
